# Learning to split
***
Is it possible to teach a model to split a rule-of-thumb into action and judgement?\
Task:
* Load and parse the Social-Chem-101 dataset consisting of ~300k rules of thumb, which were manually split into action and judgment
* Train a seq2seq model on generating the sentence judgement + action from a given rot
* Use the model to split the complete moral stories dataset, probably in another notebook though!

In [3]:
import pandas as pd
import numpy as np
import string

pd.set_option('display.max_colwidth', 400)

from ailignment.datasets import get_social_chem101

In [4]:
data = get_social_chem101().sample(100)
# do some data cleaning
data.dropna(inplace=True)

# add punctuation to each rot-judgment
data["rot-judgment"] = data["rot-judgment"].apply(lambda x: x if x[-1] in ".!" else x+".")

data["targets"] = data["rot-judgment"] + " " + data["action"]

data[["rot-judgment", "rot", "action"]]

,rot-judgment,rot,action
235665,It's okay.,It is ok to express your feelings.,expressing your feelings.
93605,bad.,It is bad to be a freeloader,being a freeloader
53588,You can't.,You can't expect your partner to stay with you.,expecting your partner to stay with you.
268972,It's good.,It's good to be honest.,being honest.
199902,It is wrong.,It is wrong to be overbearing in your adult sons life.,being overbearing in your adult sons life.
...,...,...,...
220092,should not.,You should not be wasteful.,being wasteful.
75376,should.,You should apologize to someone if you've hurt them.,apologizing to someone if you've hurt them.
95345,You should.,You should be able to do whatever activity you want,being able to do whatever activity you want
91165,should.,You should not use the pictures of others to gratify yourself without consent.,not using the pictures of others to gratify yourself without consent.


In [6]:
# load the NLI model and its tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(name)
model = AutoModelForSeq2SeqLM.from_pretrained(name)

In [7]:
# fix up huggingface dataset
from datasets import Dataset
from sklearn.model_selection import train_test_split

def tok_inp(x):
    return tokenizer(x["rot"], padding="max_length", truncation=True)

def tok_out(x):
    with tokenizer.as_target_tokenizer():
        y = tokenizer(x["targets"], padding="max_length", truncation=True)
    return y

def preprocess(dataframe):
    dataset = Dataset.from_pandas(dataframe)
    dataset =  dataset.map(tok_inp, batched=True)
    labels = dataset.map(tok_out, batched=True)
    dataset = dataset.add_column("labels", labels["input_ids"])
    return dataset

train, test = train_test_split(data, test_size=0.1)

train_data = preprocess(train)
eval_data = preprocess(test)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

import torch

data_collator = DataCollatorForSeq2Seq(tokenizer, model)

training_args = Seq2SeqTrainingArguments(
    output_dir="results/garbo/",
    num_train_epochs=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    evaluation_strategy="epoch",
    save_steps=10000,
)

In [9]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: action-char-involved, rot-categorization, rot-judgment, __index_level_0__, breakdown-worker-id, action-moral-judgment, rot, action, area, action-legal, rot-char-targeting, rot-worker-id, m, situation, characters, rot-moral-foundations, situation-short-id, targets, rot-bad, n-characters, action-agree, rot-id, action-pressure, split, rot-agree, action-hypothetical, action-agency.
***** Running training *****
  Num examples = 57
  Num Epochs = 10
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 80
/home/kiehne/workspace/jupyter-notebooks/env/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return 

Epoch,Training Loss,Validation Loss
1,No log,16.666628
2,No log,16.472795
3,No log,16.154345
4,No log,15.696165


The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: action-char-involved, rot-categorization, rot-judgment, __index_level_0__, breakdown-worker-id, action-moral-judgment, rot, action, area, action-legal, rot-char-targeting, rot-worker-id, m, situation, characters, rot-moral-foundations, situation-short-id, targets, rot-bad, n-characters, action-agree, rot-id, action-pressure, split, rot-agree, action-hypothetical, action-agency.
***** Running Evaluation *****
  Num examples = 7
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: action-char-involved, rot-categorization, rot-judgment, __index_level_0__, breakdown-worker-id, action-moral-judgment, rot, action, area, action-legal, rot-char-targeting, rot-worker-id, m, situation, characters, rot-moral-foundations, situation-short-id, targets,

KeyboardInterrupt: 

In [7]:
path = "../data/models/learn_to_split"
model.save_pretrained(path)
tokenizer.save_pretrained(path)

Configuration saved in ../data/models/learn_to_split\config.json
Model weights saved in ../data/models/learn_to_split\pytorch_model.bin
tokenizer config file saved in ../data/models/learn_to_split\tokenizer_config.json
Special tokens file saved in ../data/models/learn_to_split\special_tokens_map.json


('../data/models/learn_to_split\\tokenizer_config.json',
 '../data/models/learn_to_split\\special_tokens_map.json',
 '../data/models/learn_to_split\\tokenizer.json')

In [ ]:
tokens = tokenizer("You should not. hurting your siblings.", add_special_tokens=True)["input_ids"]
print(tokens)
print(tokenizer.decode(tokens))

In [8]:
hard_samples = [291372, 291975, 270432, 216267, 287788 ]
sample = data.sample(10).copy()
sample = data.loc[hard_samples]
x = tokenizer(sample["rot"].to_list(), padding="max_length", return_tensors="pt")
x = {k:v.cuda() for k,v in x.items()}
y = model.generate(**x, do_sample=True, min_length=1, max_length=100, top_p=0.95, top_k=50, num_beams=10, temperature=1.0)
sample["split"] = tokenizer.batch_decode(y, skip_special_tokens=True)
sample[["rot","split", "action"]]

c:\users\nikla\workspace\piep\env\lib\site-packages\torch\_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ..\aten\src\ATen\native\BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


,rot,split,action
291372,"It is appropriate to contact child services if you know of a child being mentally, physically, or sexually abused.","It is appropriate. contacting child services if you know of a child being mentally, physically, or sexually abused.","contacting child services if you know of a child being mentally, physically, or sexually abused."
291975,It's expected that family members should stick together.,It's expected. family members sticking together.,sticking together with family members.
270432,"If you think you're sick, you should see a doctor.",should. seeing a doctor if you think you're sick,seeing a doctor when you think you're sick.
216267,It's better to get a job after high school and not head straight to college.,It's better. getting a job after high school and not head straight to college.,getting a job after high school and not heading straight to college.
287788,It's okay to be annoyed when it takes the repair shop a long time to get you taken care of.,It's okay. being annoyed when it takes the repair shop a long time to get you taken care of.,being annoyed when it takes the repair shop a long time to get you taken care of.


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("../data/models/learn_to_split/").cuda()